In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [3]:
# read table

dataPayment = '../Datasets/Employee_Payroll.csv'
payroll = pd.read_csv(dataPayment)

# set default value to 0 for NaN numerical data
numeric_cols = payroll.select_dtypes(include=[np.number]).columns
payroll[numeric_cols] = payroll[numeric_cols].clip(lower=0)


payroll.fillna(0, inplace=True)
payroll['Office'] = payroll['Office'].astype(int)

# define column for 1/4 year discretization
payroll['Fiscal Quarter'] = payroll['Fiscal Quarter']*0.25 - 0.25
payroll['Fiscal Period'] = payroll['Fiscal Year'] + payroll['Fiscal Quarter']

pd.to_datetime(payroll['Original Hire Date'])

# parse hire date to get hire year
payroll['Original Hire Date'] = payroll['Original Hire Date'].str.split('/').str[2]
payroll['Original Hire Date'] = payroll['Original Hire Date'].astype(int)

_work_year = payroll["Fiscal Period"] - payroll['Original Hire Date']

In [4]:
# specify table column included

payroll = payroll[['Fiscal Year', 'Fiscal Period', 'Job Code', 'Job Title', 'Base Pay', 'Position ID', 'Employee Identifier']]

# add Working Year Column
payroll["Working Year"] = _work_year

payroll.head(20)

,Fiscal Year,Fiscal Period,Job Code,Job Title,Base Pay,Position ID,Employee Identifier,Working Year
0,2016,2016.00,1172,Assistant State's Attorney,20088.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.00
1,2016,2016.25,1172,Assistant State's Attorney,23436.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.25
2,2016,2016.50,1172,Assistant State's Attorney,20422.82,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.50
3,2016,2016.75,1172,Assistant State's Attorney,23904.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.75
4,2017,2017.00,1172,Assistant State's Attorney,20745.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,12.00
5,2017,2017.25,1172,Assistant State's Attorney,24473.38,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,12.25
6,2017,2017.50,1172,Assistant State's Attorney,21217.35,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,12.50
7,2016,2016.00,5049,Residential Model Sr Anal III,17770.86,9500731,f313b1c3-1b1a-4b07-bb75-a8c850a91bac,18.00
8,2016,2016.25,5049,Residential Model Sr Anal III,20800.67,9500731,f313b1c3-1b1a-4b07-bb75-a8c850a91bac,18.25
9,2016,2016.50,5049,Residential Model Sr Anal III,17873.76,9500731,f313b1c3-1b1a-4b07-bb75-a8c850a91bac,18.50


In [5]:
# YANG INI JANGAN DIJALANKAN DULU YA GESSS :'

arr = []

try:
    with open('../cache/ids.txt', "r") as f:
        for _id in f:
            arr.append(int(_id))
except:
    if not os.path.exists('../cache'):
        os.mkdir('../cache')
        
    _index = payroll['Job Code'].unique()
    for _id in _index:
        counts = len(payroll.groupby(['Job Code']).get_group(_id))
        if counts > 1000:
            arr.append(str(_id))
    with open('../cache/ids.txt', 'w') as f:
        for _id in arr:
            f.write('%s\n' % _id) 
finally:
   f.close()

# :return : <List> arr : list of unique job id

In [6]:
# Group Job Code 1172 training dataset <<< PAKAI INI NGGIH

group_1172 = payroll.groupby(['Job Code']).get_group(1172)

_sorted = group_1172.sort_values('Employee Identifier')
fiscal = pd.pivot_table(_sorted, values='Base Pay', index=['Fiscal Period'], columns='Employee Identifier')

fiscal = fiscal.diff()

In [7]:
# cleaned display table

fiscal.fillna(0, inplace=True)
fiscal = fiscal.drop(2016.00)

fiscal

Employee Identifier,007ca4de-de2b-44ac-bf71-6eb4cbdc27e2,00d20c14-93d4-4560-a10d-3e41c1fafd07,00d3e7d2-9397-4a16-a556-80056ebbf456,00e833c8-ecbf-4e2c-9b66-9472d1949d79,01143446-4dfb-4327-840d-fb8b88fe78c3,0163b34b-b1d9-4b9c-aa9e-90df3b8fd9af,017d1eb1-6574-48f1-9bd9-536eb23a2219,019f5fbb-4afd-4bd7-a254-eb03660a979f,01ee96aa-5bdb-4694-b3cb-be2a2960a4d2,028d2bca-7dab-4294-a9fc-f4371956438b,...,feafd46f-ba8a-41c0-b72e-0cc961d66b01,feb07f2b-e7cd-40ec-9b95-103ef2246d6c,febdb26a-8424-438a-8a5d-fbc3c17d1ab5,fecb6d8c-da1b-4752-a4d4-14de89f24c46,feccda27-41c4-475e-8f78-cc16adff92b2,fef3a6ad-266b-44b5-96f5-2952dc364b38,ff131ab6-ce88-43ad-b6c1-64507471dc08,ff3dd422-b4e5-4b17-87be-23a558efb62e,ff9f0a14-2aa1-4ff5-8ad2-8b4db9a3d719,ffa8ef63-a6ed-49c2-8b32-f4bbbba79851
Fiscal Period,,,,,,,,,,,,,,,,,,,,,
2016.25,2688.250,2583.05,9540.17,4253.60,3717.52,2688.26,3888.34,2825.88,4630.00,3201.11,...,0.00,3644.26,4746.800,2532.08,0.00,3644.25,2482.09,5014.51,2482.08,4630.00
2016.50,-2419.420,-2324.64,-2146.63,-3828.40,-3345.92,-2419.45,-3499.50,-2543.44,-4166.80,-2881.12,...,0.00,-6996.95,-18372.955,-2278.90,0.00,-3279.82,-2233.67,-4512.94,-2233.67,-13612.13
2016.75,-2688.240,2686.40,-2385.03,4423.65,3866.11,2795.78,4043.84,2938.80,4815.27,3329.10,...,0.00,0.00,24313.385,2633.37,0.00,3790.00,2581.46,5215.19,2581.46,0.00
2017.00,0.000,-2437.13,0.00,-4013.40,-3507.40,-2536.41,-3668.49,-241.64,-4368.65,-3020.29,...,0.00,0.00,-5175.780,-2230.73,0.00,-3438.29,-2341.77,-4731.29,-2341.76,0.00
2017.25,0.000,2713.77,0.00,4468.71,3905.59,2824.22,4652.95,3379.89,5055.94,3363.05,...,0.00,0.00,5763.290,2871.37,28846.15,3828.65,2762.79,5268.32,2697.86,0.00
2017.50,0.000,-2674.24,0.00,-4403.73,-3848.70,-2783.11,-4106.70,-3330.66,-4697.60,-3314.06,...,10272.24,0.00,-5679.360,-2674.24,-5769.23,-3772.90,-2414.77,-5191.60,-2582.67,0.00
2017.75,0.000,2455.30,0.00,4132.10,3844.49,2504.83,3696.04,3192.25,4357.24,2982.66,...,3361.82,0.00,5111.400,2406.82,5192.31,3615.54,2359.24,4672.44,2324.39,0.00
2018.00,0.000,-1925.43,0.00,0.00,-3378.40,-159.66,-3593.23,-2789.46,-18881.30,-2982.66,...,-3174.22,0.00,-4969.360,-2040.58,-28269.23,-2484.36,-2161.45,-4280.44,-2195.54,0.00
2018.25,14593.945,2810.75,0.00,0.00,3926.40,4270.59,4730.22,3397.75,2469.30,3314.08,...,4265.44,4147.82,5736.140,2974.37,0.00,3172.57,2914.84,5348.40,2949.53,2420.68


In [ ]:


# neg_cols = (payroll[numeric_cols] < 0).any()

# if neg_cols.any():
#     print("Terdapat nilai negatif pada kolom: ", end="")
#     print(", ".join(neg_cols[neg_cols == True].index))
# else:
#     print("Tidak terdapat nilai negatif pada semua kolom numerik.")

In [ ]:
# SUDAH AMAN 👍👍

# null_cols = payroll.isnull().any()

# if null_cols.any():
#     print("Terdapat nilai NaN pada kolom: ", end="")
#     print(", ".join(null_cols[null_cols == True].index))
# else:
#     print("Tidak terdapat nilai NaN pada semua kolom.")

In [ ]:
# query job code indexing

for val in arr:
    new_df = payroll[payroll['Job Code'] == val]
    print(f"DataFrame for job code {val}:")
    display(new_df)